In [1]:
from __future__ import print_function, division, absolute_import

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7f9347ec34e0>

In [3]:
with open('./me_models/iBS.pickle', 'rb') as f:
    me = pickle.load(f)

In [4]:
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.lower_bound = 0
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.upper_bound = 0
## Homogenize exchange reactions
for rxn in me.reactions:
    if 'EX_' in rxn.id:
        try:
            lb = bsub.reactions.get_by_id(rxn.id).lower_bound
            ub = bsub.reactions.get_by_id(rxn.id).upper_bound
            rxn.lower_bound = lb
            rxn.upper_bound = ub
        except:
            print(rxn.id, 'not in m_model')

        

EX_4fe4s_c not in m_model
EX_2fe2s_c not in m_model
EX_udcpp_c not in m_model
EX_mn2_c not in m_model
EX_pydx5p_c not in m_model


In [5]:
solve_me_model(me, 1., min_mu = .1, precision=1e-2, using_soplex=False)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 95.181796 seconds
Finished substituting S,lb,ub in 3.663058 seconds
Finished makeME_LP in 0.823270 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.5 0.5 1
Finished substituting S,lb,ub in 3.274899 seconds
Finished makeME_LP in 0.660914 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 3.239850 seconds
Finished makeME_LP in 0.664447 seconds
Getting MINOS parameters from ME_NLP...
3 0.125 0.125 0.25 0.125 optimal
Finished substituting S,lb,ub in 3.164641 seconds
Finished makeME_LP in 0.659676 seconds
Getting MINOS parameters from ME_NLP...
4 0.125 0.125 0.1875 0.1875 1
Finished substituting S,lb,ub in 3.137505 seconds
Finished makeME_LP in 0.658968 seconds
Getting MINOS parameters from ME_NLP...
5 0.125 0.125 0.15625 0.15625 1
Finished substituting S,lb,ub in 3.269128 seconds
Finished makeME_LP in 0.752199 seconds
Getting MINOS parameters from M

In [6]:
with open('./me_models/solution.pickle', 'wb') as solution:
    pickle.dump(me,solution)

In [2]:
with open('./me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

In [7]:
df = exchange_single_model(me, solution = 0)

In [8]:
df.

,id,name,reaction,lower_bound,upper_bound,flux
0,EX_co2_e,,co2_e <=>,-999999.0,999999.0,5.140316e+00
1,EX_glc__D_e,,glc__D_e <=>,-1.7,999999.0,-1.654613e+00
2,EX_h2o_e,,h2o_e <=>,-999999.0,999999.0,7.400143e+00
3,EX_h_e,,h_e <=>,-999999.0,999999.0,1.996924e-01
4,EX_mg2_e,,mg2_e <=>,-999999.0,999999.0,-3.643748e-04
5,EX_nh4_e,,nh4_e <=>,-5.0,999999.0,-7.079520e-01
6,EX_o2_e,,o2_e <--,-999999.0,0.0,-4.687243e+00
7,EX_pi_e,,pi_e <=>,-5.0,999999.0,-4.891938e-01
8,EX_so4_e,,so4_e <=>,-5.0,999999.0,-1.147857e-02
9,EX_2fe2s_c,,2fe2s_c <=>,-1000.0,1000.0,-2.941439e-08


In [9]:
df_m = exchange_single_model(bsub, solution = 0)

In [10]:
df_m

,id,name,reaction,lower_bound,upper_bound,flux
0,EX_ca2_e,Calcium exchange,ca2_e <=>,-999999.0,999999.0,-0.000378
1,EX_co2_e,CO2 exchange,co2_e <=>,-999999.0,999999.0,5.980676
2,EX_fe3_e,Fe3+ exchange,fe3_e <=>,-999999.0,999999.0,-0.000407
3,EX_glc__D_e,D-Glucose exchange,glc__D_e <=>,-1.7,999999.0,-1.700000
4,EX_h2o_e,H2O exchange,h2o_e <=>,-999999.0,999999.0,7.869819
5,EX_h_e,H+ exchange,h_e <=>,-999999.0,999999.0,0.767665
6,EX_k_e,K+ exchange,k_e <=>,-999999.0,999999.0,-0.083321
7,EX_mg2_e,Mg exchange,mg2_e <=>,-999999.0,999999.0,-0.011999
8,EX_nh4_e,Ammonia exchange,nh4_e <=>,-5.0,999999.0,-0.944964
9,EX_o2_e,O2 exchange,o2_e <--,-999999.0,0.0,-5.705899


In [11]:
filename = 'solution_summary.csv'
file = open(filename,'w')
file.write('ID, reaction, LB, UB, V \n')
for rxn in me.reactions:
    try:
        reaction = rxn.reaction
    except:
        reaction = 'NA'
    string = rxn.id + ',' + reaction + ',' + str(rxn.lower_bound) + ',' + str(rxn.upper_bound) +\
    ',' +  str(me.solution.x_dict[rxn.id]) + '\n'
    file.write(string)
file.close()

In [ ]:
# Display metabolic flux on escher map
show_escher_map(me).display_in_notebook()

In [12]:
flux_dict = me.get_metabolic_flux()

In [13]:
filename = 'fluxdist_bsub.csv'
file = open(filename,'w')
for rxn in flux_dict.keys():
    flux = flux_dict[rxn]
    file.write(rxn + ',' + str(flux) + '\n')
file.close()

In [ ]:
filename = 'fluxdist_m_bsub.csv'
file = open(filename,'w')
for rxn in bsub.reactions:
    flux = bsub.solution.x_dict[rxn.id]
    file.write(rxn.id + ',' + str(flux) + '\n')
file.close()